In [193]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import requests
import numpy as np
import time
import os
import time
from requests import get


def get_amino_acid_sequence(uniprot_id):
    base_url = "https://www.uniprot.org/uniprot/"
    response_format = ".fasta"

    # Combine the URL to get the FASTA format data for the given UniProt ID
    url = f"{base_url}{uniprot_id}{response_format}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (e.g., 404, 500)

        # Parse the response to extract the amino acid sequence
        lines = response.text.strip().split("\n")
        sequence = "".join(lines[1:])  # Skipping the first line (header)

        return sequence
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching the data: {e}")
        return None
    


In [202]:


def get_top_e_val(uniprot_id, max_retries=10):
    """
    Performs a BLAST search against the NCBI nr database for the given UniProt ID.
    Returns the top hit's e-value and score.

    :param uniprot_id: The UniProt ID to search for.
    :param max_retries: The maximum number of times to retry the BLAST search if it fails.
    """



    query_sequence = get_amino_acid_sequence(uniprot_id)
    hits = []

    #values to return
    cerevisiae_e_val = np.nan
    cerevisiae_score = np.nan
    cerevisiae_symbol = "N/A"
    albicans_e_val = np.nan
    albicans_score = np.nan
    albicans_symbol = "N/A"

    for retry in range(max_retries + 1):
        try:
            # Perform the BLAST search
            print("BLAST starts")



            result_handle = NCBIWWW.qblast("blastp", "nr", query_sequence, expect=1000, entrez_query="txid237561[ORGN] OR txid5476[ORGN] OR txid559292[ORGN] OR txid4932[ORGN]",hitlist_size=1000)

            blast_records = NCBIXML.parse(result_handle)

            hits = []
            for blast_record in blast_records:
                for alignment in blast_record.alignments:
                    for hsp in alignment.hsps:
                        hit_info = {
                                        "accession": alignment.accession,
                                        "description": alignment.title,
                                        "evalue": hsp.expect,
                                        "score": hsp.score
                                    }
                        hits.append(hit_info)
            #in hits find entries, where description contains the string "Saccharomyces cerevisiae"
            cerevisiae_entries = [hit for hit in hits if "Saccharomyces cerevisiae" in hit["description"]]
            albicans_entries = [hit for hit in hits if "Candida albicans" in hit["description"]]
            #order these by hgihest score first lowest score last
            cerevisiae_entries = sorted(cerevisiae_entries, key=lambda x: x["score"], reverse=True)
            albicans_entries = sorted(albicans_entries, key=lambda x: x["score"], reverse=True)

            if len(cerevisiae_entries) > 0:
                cerevisiae_e_val = cerevisiae_entries[0]["evalue"]
                cerevisiae_score = cerevisiae_entries[0]["score"]
                cerevisiae_symbol = cerevisiae_entries[0]["accession"]
            
            
            if len(albicans_entries) > 0:
                albicans_e_val = albicans_entries[0]["evalue"]
                albicans_score = albicans_entries[0]["score"]
                albicans_symbol = albicans_entries[0]["accession"]

            return cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol

        except Exception as e:
            print(f"Error during attempt {retry + 1}: {e}")
            if retry < max_retries:
                print("Retrying...")
                time.sleep(60)  # Add a delay before retrying
            else:
                print(f"Max retries reached, returning default values.")
                return cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol


In [206]:
#import pombase gene products file
import pandas as pd
import numpy as np
root = "/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/"
gene_info = pd.read_csv(root + "external data/Pombase files/gene_IDs_names_products.tsv", sep='\t', header=0)
gene_info = gene_info[["SPAC1002.01", "Q9US57", "mrx11"]]
gene_info = gene_info.rename(columns={"SPAC1002.01": "ID", "Q9US57": "uniprot", "mrx11":"gene_name"})

orthology_db = pd.read_csv(root + "Bence folder/Orthologs/final_orthology_database.csv")
#fetch the list of pombe_gene_IDs with orthopattern 7 and 1 into lists called conserved, unique
unique_genes= orthology_db[orthology_db['orthopattern'] == 1]["pombe_name(s)"].values
unique_genes = np.array([element[2:-2] for element in unique_genes])
unique_genes_uniprot = np.array([])
to_be_deleted = np.array([],dtype=int)
for i in range(len(unique_genes)):
    #find in gene_info the uniprot ID of the gene
    try:
        unique_genes_uniprot = np.append(unique_genes_uniprot, gene_info[gene_info["ID"] == unique_genes[i]]["uniprot"].values[0])
    except:
        unique_genes_uniprot = unique_genes_uniprot
        #remove that entry from unique_genes
        to_be_deleted = np.append(to_be_deleted, i)
unique_genes = np.delete(unique_genes, to_be_deleted)



conserved_genes= orthology_db[orthology_db['orthopattern'] == 7]["pombe_name(s)"].values
conserved_genes = np.array([element[2:-2] for element in conserved_genes])
#take uniprot IDs but in the same order!
conserved_genes_uniprot = np.array([])
to_be_deleted = np.array([],dtype=int)
for i in range(len(conserved_genes)):
    try:
        conserved_genes_uniprot = np.append(conserved_genes_uniprot, gene_info[gene_info["ID"] == conserved_genes[i]]["uniprot"].values[0])
    except:
        conserved_genes_uniprot = conserved_genes_uniprot
        to_be_deleted = np.append(to_be_deleted, i)
conserved_genes = np.delete(conserved_genes, to_be_deleted)



In [6]:
conserved_e_vals_cerevisiae = np.full(50, np.nan)
conserved_scores_cerevisiae = np.full(50, np.nan)
conserved_e_vals_albicans = np.full(50, np.nan)
conserved_scores_albicans = np.full(50, np.nan)
conserved_symbols_cerevisiae = np.full(50, "N/A")
conserved_symbols_albicans = np.full(50, "N/A")

In [208]:

try:
    conserved_e_vals_cerevisiae = np.loadtxt('conserved_e_vals_cerevisiae.txt')
    #drop all values after the last non nan value
    #n_entries = np.where(~np.isnan(conserved_e_vals_cerevisiae))[0][-1]
    n_entries=0
    conserved_e_vals_cerevisiae = conserved_e_vals_cerevisiae[:n_entries]
    #fill the rest with nan
    conserved_e_vals_cerevisiae = np.append(conserved_e_vals_cerevisiae, np.full(50-n_entries, np.nan))
    print("file loaded, entries:", n_entries)

except:
    conserved_e_vals_cerevisiae = np.full(50, np.nan)

try:
    conserved_scores_cerevisiae = np.loadtxt('conserved_scores_cerevisiae.txt')
    conserved_scores_cerevisiae = conserved_scores_cerevisiae[:n_entries]
    conserved_scores_cerevisiae = np.append(conserved_scores_cerevisiae, np.full(50-n_entries, np.nan))
    print("file loaded, entries:",n_entries)
except:
    conserved_scores_cerevisiae = np.full(50, np.nan)

try:
    conserved_e_vals_albicans = np.loadtxt('conserved_e_vals_albicans.txt')
    conserved_e_vals_albicans = conserved_e_vals_albicans[:n_entries]
    conserved_e_vals_albicans = np.append(conserved_e_vals_albicans, np.full(50-n_entries, np.nan))
    print("file loaded, entries:", n_entries)
except:
    conserved_e_vals_albicans = np.full(50, np.nan)

try:
    conserved_scores_albicans = np.loadtxt('conserved_scores_albicans.txt')
    conserved_scores_albicans = conserved_scores_albicans[:n_entries]
    conserved_scores_albicans = np.append(conserved_scores_albicans, np.full(50-n_entries, np.nan))
    print("file loaded, entries:", n_entries)

except:
    conserved_scores_albicans = np.full(50, np.nan)

try:
    conserved_symbols_cerevisiae = np.loadtxt('conserved_symbols_cerevisiae.txt', dtype=str)
    conserved_symbols_cerevisiae = conserved_symbols_cerevisiae[:n_entries]
    conserved_symbols_cerevisiae = np.append(conserved_symbols_cerevisiae, np.full(50-n_entries, "N/A"))
    print("file loaded, entries:", len(n_entries))

except:
    conserved_symbols_cerevisiae = np.full(50, "N/A")

try:
    conserved_symbols_albicans = np.loadtxt('conserved_symbols_albicans.txt', dtype=str)
    conserved_symbols_albicans = conserved_symbols_albicans[:n_entries]
    conserved_symbols_albicans = np.append(conserved_symbols_albicans, np.full(50-n_entries, "N/A"))
    print("file loaded, entries:", len(n_entries))

except:
    conserved_symbols_albicans = np.full(50, "N/A")



for i in range(n_entries,50):

    cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol = get_top_e_val(conserved_genes_uniprot[i]) 
    print(cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol,conserved_genes[i],conserved_genes_uniprot[i])
    print("currently at entry", i, "of", len(conserved_genes_uniprot))
    conserved_e_vals_cerevisiae[i] = cerevisiae_e_val
    conserved_scores_cerevisiae[i] = cerevisiae_score
    conserved_symbols_cerevisiae[i] = cerevisiae_symbol
    conserved_e_vals_albicans[i] = albicans_e_val
    conserved_scores_albicans[i] = albicans_score
    conserved_symbols_albicans[i] = albicans_symbol




file loaded, entries: 0
file loaded, entries: 0
file loaded, entries: 0
file loaded, entries: 0
BLAST starts
9.5634e-26 299.0 EEU05955 nan nan N/A SPBC336.07 Q9UST7
currently at entry 0 of 1070
BLAST starts
6.96575e-49 408.0 CAD6595082 2.69148e-54 445.0 KGU29437 SPCC1450.04 O74173
currently at entry 1 of 1070
BLAST starts
1.64219e-42 399.0 AJO98514 8.05207e-43 400.0 RLP63156 SPAC17H9.17c O13814
currently at entry 2 of 1070
BLAST starts
0.0 2272.0 CAI4237107 0.0 2253.0 EEQ42403 SPAC56F8.03 Q10251
currently at entry 3 of 1070
BLAST starts
2.50968e-21 209.0 AJO93589 3.72198e-23 222.0 KHC56833 SPCC4B3.11c Q9USK1
currently at entry 4 of 1070
BLAST starts
2.74025e-68 573.0 CAI4238260 6.50536e-107 833.0 KGU27560 SPAC23H3.05c O42858
currently at entry 5 of 1070
BLAST starts
1.64664e-136 1066.0 AJO97954 1.18354e-127 1007.0 KGU30207 SPBC660.13c Q92372
currently at entry 6 of 1070
BLAST starts
2.67218e-10 165.0 CAI4239675 3.96087e-05 122.0 KHC65636 SPBC1706.01 O60132
currently at entry 7 of 1070


In [209]:

np.savetxt('conserved_e_vals_cerevisiae_final.txt', conserved_e_vals_cerevisiae)
np.savetxt('conserved_e_vals_albicans_final.txt', conserved_e_vals_albicans)
np.savetxt('conserved_symbols_cerevisiae_final.txt', conserved_symbols_cerevisiae, fmt="%s")
np.savetxt('conserved_scores_cerevisiae_final.txt', conserved_scores_cerevisiae)
np.savetxt('conserved_scores_albicans_final.txt', conserved_scores_albicans)
np.savetxt('conserved_symbols_albicans_final.txt', conserved_symbols_albicans, fmt="%s")


In [213]:
np.savetxt('unique_e_vals_cerevisiae_final.txt', unique_e_vals_cerevisiae)
np.savetxt('unique_e_vals_albicans_final.txt', unique_e_vals_albicans)
np.savetxt('unique_scores_cerevisiae_final.txt', unique_scores_cerevisiae)
np.savetxt('unique_scores_albicans_final.txt', unique_scores_albicans)
np.savetxt('unique_symbols_cerevisiae_final.txt', unique_symbols_cerevisiae, fmt="%s")
np.savetxt('unique_symbols_albicans_final.txt', unique_symbols_albicans, fmt="%s")

In [212]:

try:
    unique_e_vals_cerevisiae = np.loadtxt('unique_e_vals_cerevisiae_final.txt')
    #drop all values after the last non nan value
    n_entries = np.where(~np.isnan(unique_e_vals_cerevisiae))[0][-1]
    
    unique_e_vals_cerevisiae = unique_e_vals_cerevisiae[:n_entries]
    #fill up with nans
    unique_e_vals_cerevisiae = np.append(unique_e_vals_cerevisiae, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)

except:
    unique_e_vals_cerevisiae = np.array([])

try:
    unique_scores_cerevisiae = np.loadtxt('unique_scores_cerevisiae_final.txt')
    unique_scores_cerevisiae = unique_scores_cerevisiae[:n_entries]
    unique_scores_cerevisiae = np.append(unique_scores_cerevisiae, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)
except:
    unique_scores_cerevisiae = np.array([])

try:
    unique_e_vals_albicans = np.loadtxt('unique_e_vals_albicans_final.txt')
    unique_e_vals_albicans = unique_e_vals_albicans[:n_entries]
    unique_e_vals_albicans = np.append(unique_e_vals_albicans, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)
except:
    unique_e_vals_albicans = np.array([])

try:
    unique_scores_albicans = np.loadtxt('unique_scores_albicans_final.txt')
    unique_scores_albicans = unique_scores_albicans[:n_entries]
    unique_scores_albicans = np.append(unique_scores_albicans, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)
except:
    unique_scores_albicans = np.array([])



try:
    unique_symbols_cerevisiae = np.loadtxt('unique_symbols_cerevisiae_final.txt', dtype=str)
    unique_symbols_cerevisiae = unique_symbols_cerevisiae[:n_entries]
    unique_symbols_cerevisiae = np.append(unique_symbols_cerevisiae, np.full(len(unique_genes_uniprot)-n_entries, "N/A"))
    print("file loaded, entries:", n_entries)

except:
    unique_symbols_cerevisiae = np.full(len(unique_genes_uniprot), "N/A")

try:
    unique_symbols_albicans = np.loadtxt('unique_symbols_albicans_final.txt', dtype=str)
    unique_symbols_albicans = unique_symbols_albicans[:n_entries]
    unique_symbols_albicans = np.append(unique_symbols_albicans, np.full(len(unique_genes_uniprot)-n_entries, "N/A"))
    print("file loaded, entries:", n_entries)

except:
    unique_symbols_albicans = np.full(len(unique_genes_uniprot), "N/A")




for i in range(n_entries,len(unique_genes_uniprot)):

    cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol = get_top_e_val(unique_genes_uniprot[i]) 
    print(cerevisiae_e_val, cerevisiae_score, cerevisiae_symbol, albicans_e_val, albicans_score, albicans_symbol,unique_genes[i],unique_genes_uniprot[i])
    print("currently at entry", i, "of", len(unique_genes_uniprot))
    unique_e_vals_cerevisiae[i] = cerevisiae_e_val
    unique_scores_cerevisiae[i] = cerevisiae_score
    unique_symbols_cerevisiae[i] = cerevisiae_symbol
    unique_e_vals_albicans[i] = albicans_e_val
    unique_scores_albicans[i] = albicans_score
    unique_symbols_albicans[i] = albicans_symbol


np.savetxt('unique_e_vals_cerevisiae_final.txt', unique_e_vals_cerevisiae)
np.savetxt('unique_e_vals_albicans_final.txt', unique_e_vals_albicans)
np.savetxt('unique_scores_cerevisiae_final.txt', unique_scores_cerevisiae)
np.savetxt('unique_scores_albicans_final.txt', unique_scores_albicans)
    


file loaded, entries: 16
file loaded, entries: 16
file loaded, entries: 16
file loaded, entries: 16
file loaded, entries: 16
file loaded, entries: 16
BLAST starts
7.45392 73.0 CAI4823182 nan nan N/A SPBC359.04c Q9P5N1
currently at entry 16 of 25
BLAST starts
nan nan N/A 618.52 58.0 KAF6070001 SPBC646.06c O94510
currently at entry 17 of 25
BLAST starts
nan nan N/A nan nan N/A SPBC947.04 Q874R4
currently at entry 18 of 25
BLAST starts
0.00986924 109.0 CAI6684116 7.16111e-62 607.0 AAQ03243 SPBPJ4664.02 Q96WV6
currently at entry 19 of 25
BLAST starts
nan nan N/A nan nan N/A SPCC1223.13 O74412
currently at entry 20 of 25
BLAST starts
0.000244881 119.0 CAI4240790 2.27603e-05 127.0 AAN73329 SPCC1742.01 Q9P6S0
currently at entry 21 of 25
BLAST starts
nan nan N/A 0.000183926 115.0 O74623 SPCC188.09c Q7Z9I1
currently at entry 22 of 25
BLAST starts
115.489 66.0 CAD6645984 nan nan N/A SPCC736.08 O74954
currently at entry 23 of 25
BLAST starts
7.41169e-30 336.0 CAI7443233 3.17752e-33 364.0 KGT71740

In [ ]:
np.savetxt('unique_e_vals_cerevisiae_final.txt', unique_e_vals_cerevisiae)
np.savetxt('unique_e_vals_albicans_final.txt', unique_e_vals_albicans)
np.savetxt('unique_scores_cerevisiae_final.txt', unique_scores_cerevisiae)
np.savetxt('unique_scores_albicans_final.txt', unique_scores_albicans)

In [ ]:
import time
import subprocess
#import get
from requests import get

def scores_evals_from_uniprot(uniprot_id):
    """
    Returns the e-value and score of the best hit in S. cerevisiae and C. albicans for a given uniprot id
    
    Parameters
    ----------
    uniprot_id : str
        Uniprot id of the protein of interest
    """


    download_pdb_alphafold(uniprot_id)
    file = f'/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci\ Bahler\ lab/S.-Pombe-MLPs\ -\ Github/Bence\ folder/Structure_and_Sequence_comparisons/{uniprot_id}.pdb'
    
    x=0
    
    while x < 5:
        try:
            job = !curl -X POST -F q=@{file} -F 'mode=3diaa' -F 'database[]=afdb-swissprot' https://search.foldseek.com/api/ticket
            job = job[-1].split('"')[3]
            result = get('https://search.foldseek.com/api/result/' + job + '/0').json()
            x = 5
        except:
            time.sleep(60)
            x += 1
            print("waiting")
           
    try:
        taxids = [result["results"][0]["alignments"][i]["taxId"] for i in range(len(result["results"][0]["alignments"]))]
        scores = [result["results"][0]["alignments"][i]["score"] for i in range(len(result["results"][0]["alignments"]))]
        evals = [result["results"][0]["alignments"][i]["eval"] for i in range(len(result["results"][0]["alignments"]))]

        #for top_score_cerevisiae find in taxids the entry which equals 4932 or 559292 and get the highest score and evalue from those
        cerevisiae_index = [i for i, x in enumerate(taxids) if x == 4932 or x == 559292]
        albicans_index = [i for i, x in enumerate(taxids) if x == 5476 or x== 237561]
    except:
        return 0,100,0,100
    try:
        top_cerevisiae_score = max([scores[i] for i in cerevisiae_index])
        top_cerevisiae_eval = min([evals[i] for i in cerevisiae_index])
    except:
        print("no cerevisiae")
        top_cerevisiae_score = 0
        top_cerevisiae_eval = 100
        

        #do the same for albicans
        
    try:
        top_albicans_score = max([scores[i] for i in albicans_index])
        top_albicans_eval = min([evals[i] for i in albicans_index])
    except:
        print("no albicans")
        top_albicans_score = 0
        top_albicans_eval = 100
        
    

    return top_cerevisiae_score, top_cerevisiae_eval, top_albicans_score, top_albicans_eval




In [103]:
np.savetxt('unique_e_vals_cerevisiae_foldseek.txt', unique_e_vals_cerevisiae_foldseek)
np.savetxt('unique_scores_cerevisiae_foldseek.txt', unique_scores_cerevisiae_foldseek)
np.savetxt('unique_e_vals_albicans_foldseek.txt', unique_e_vals_albicans_foldseek)
np.savetxt('unique_scores_albicans_foldseek.txt', unique_scores_albicans_foldseek)

In [52]:
unique_e_vals_cerevisiae_foldseek = np.loadtxt('unique_e_vals_cerevisiae_foldseek.txt')
    #drop all values after the last non nan value
n_entries = np.where(~np.isnan(unique_e_vals_cerevisiae_foldseek))[0][-1]
unique_e_vals_cerevisiae_foldseek = unique_e_vals_cerevisiae_foldseek[:n_entries]
    #fill up the rest until len(unique_genes_uniprot) with nan
unique_e_vals_cerevisiae_foldseek = np.append(unique_e_vals_cerevisiae_foldseek, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
print("file loaded, entries:", n_entries)

file loaded, entries: 18


In [53]:
n_entries = 0 

try:
    unique_e_vals_cerevisiae_foldseek = np.loadtxt('unique_e_vals_cerevisiae_foldseek.txt')
    #drop all values after the last non nan value
    n_entries = np.where(~np.isnan(unique_e_vals_cerevisiae_foldseek))[0][-1]
    unique_e_vals_cerevisiae_foldseek = unique_e_vals_cerevisiae_foldseek[:n_entries]
    #fill up the rest until len(unique_genes_uniprot) with nan
    unique_e_vals_cerevisiae_foldseek = np.append(unique_e_vals_cerevisiae_foldseek, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)

except:
    unique_e_vals_cerevisiae_foldseek = np.full(len(unique_genes_uniprot), np.nan)

try:
    unique_scores_cerevisiae_foldseek = np.loadtxt('unique_scores_cerevisiae_foldseek.txt')
    unique_scores_cerevisiae_foldseek = unique_scores_cerevisiae_foldseek[:n_entries]
    unique_scores_cerevisiae_foldseek = np.append(unique_scores_cerevisiae_foldseek, np.full(len(unique_genes_uniprot)-n_entries, np.nan))

    print("file loaded, entries:", n_entries)
except:
    unique_scores_cerevisiae_foldseek = np.full(len(unique_genes_uniprot), np.nan)

try:
    unique_e_vals_albicans_foldseek = np.loadtxt('unique_e_vals_albicans_foldseek.txt')
    unique_e_vals_albicans_foldseek = unique_e_vals_albicans_foldseek[:n_entries]
    unique_e_vals_albicans_foldseek = np.append(unique_e_vals_albicans_foldseek, np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)
except:
    unique_e_vals_albicans_foldseek = np.full(len(unique_genes_uniprot), np.nan)

try:
    unique_scores_albicans_foldseek  = np.loadtxt('unique_scores_albicans_foldseek.txt')
    unique_scores_albicans_foldseek  = unique_scores_albicans_foldseek [:n_entries]
    unique_scores_albicans_foldseek  = np.append(unique_scores_albicans_foldseek , np.full(len(unique_genes_uniprot)-n_entries, np.nan))
    print("file loaded, entries:", n_entries)

except:
    unique_scores_albicans_foldseek = np.full(len(unique_genes_uniprot), np.nan)

try:
    for i in range(n_entries,len(unique_genes_uniprot)):
        score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans = scores_evals_from_uniprot(unique_genes_uniprot[i])
        print(score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans, unique_genes[i], unique_genes_uniprot[i])
        print("currently at entry", i, "of", len(unique_genes_uniprot), "entries top")
        unique_e_vals_cerevisiae_foldseek[i] = eval_cerevisiae
        unique_scores_cerevisiae_foldseek[i] = score_cerevisiae
        unique_e_vals_albicans_foldseek[i] = eval_albicans
        unique_scores_albicans_foldseek[i] = score_albicans

except:
    for i in range(len(unique_genes_uniprot)):
        score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans = scores_evals_from_uniprot(unique_genes_uniprot[i])
        print(score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans, unique_genes[i], unique_genes_uniprot[i])
        print("currently at entry", i, "of", len(unique_genes_uniprot), "entries bot")
        unique_e_vals_cerevisiae_foldseek[i] = eval_cerevisiae
        unique_scores_cerevisiae_foldseek[i] = score_cerevisiae
        unique_e_vals_albicans_foldseek[i] = eval_albicans
        unique_scores_albicans_foldseek[i] = score_albicans

np.savetxt('unique_e_vals_cerevisiae_foldseek.txt', unique_e_vals_cerevisiae_foldseek)
np.savetxt('unique_scores_cerevisiae_foldseek.txt', unique_scores_cerevisiae_foldseek)
np.savetxt('unique_e_vals_albicans_foldseek.txt', unique_e_vals_albicans_foldseek)
np.savetxt('unique_scores_albicans_foldseek.txt', unique_scores_albicans_foldseek)



file loaded, entries: 18
file loaded, entries: 18
file loaded, entries: 18
file loaded, entries: 18


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  571k    0  571k    0     0  2139k      0 --:--:-- --:--:-- --:--:-- 2148k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.4M    0 11.4M    0     0  7291k      0 --:--:--  0:00:01 --:--:-- 7289k


0 100 0 100 SPBC947.04 Q874R4
currently at entry 18 of 25 entries top


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   127  100   127    0     0   1831      0 --:--:-- --:--:-- --:--:--  1840
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   127  100   127    0     0   2156      0 --:--:-- --:--:-- --:--:--  2189


waiting
waiting
waiting
waiting
waiting
0 100 0 100 SPBPJ4664.02 Q96WV6
currently at entry 19 of 25 entries top


...

waiting
waiting
no cerevisiae
0 100 22 9.048 SPCC188.09c Q7Z9I1
currently at entry 22 of 25 entries top


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  380k    0  380k    0     0  2043k      0 --:--:-- --:--:-- --:--:-- 2047k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4506k    0 4506k    0     0  6472k      0 --:--:-- --:--:-- --:--:-- 6475k


waiting
waiting
waiting
no cerevisiae
no albicans
0 100 0 100 SPCC736.08 O74954
currently at entry 23 of 25 entries top


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  937k    0  937k    0     0  3946k      0 --:--:-- --:--:-- --:--:-- 3956k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.2M    0 27.2M    0     0  8082k      0 --:--:--  0:00:03 --:--:-- 8082k


waiting
waiting
waiting
0 100 0 100 SPCC895.05 O94532
currently at entry 24 of 25 entries top


In [36]:
n_entries = 0 
try:
    conserved_e_vals_cerevisiae_foldseek = np.loadtxt('conserved_e_vals_cerevisiae.txt')
    #drop all values after the last non nan value
    #n_entries = np.where(~np.isnan(conserved_e_vals_cerevisiae_foldseek))[0][-1]
    n_entries = 0
    conserved_e_vals_cerevisiae_foldseek = conserved_e_vals_cerevisiae_foldseek[:n_entries]
    print("file loaded, entries:", len(n_entries))

except:
    conserved_e_vals_cerevisiae_foldseek = np.full(50, np.nan)

try:
    conserved_scores_cerevisiae_foldseek = np.loadtxt('conserved_scores_cerevisiae.txt')
    conserved_scores_cerevisiae_foldseek = conserved_scores_cerevisiae_foldseek[:n_entries]
    print("file loaded, entries:", len(n_entries))
except:
    conserved_scores_cerevisiae_foldseek = np.full(50, np.nan)

try:
    conserved_e_vals_albicans_foldseek = np.loadtxt('conserved_e_vals_albicans.txt')
    conserved_e_vals_albicans_foldseek = conserved_e_vals_albicans_foldseek[:n_entries]
    print("file loaded, entries:", len(n_entries))
except:
    conserved_e_vals_albicans_foldseek = np.full(50, np.nan)

try:
    conserved_scores_albicans_foldseek  = np.loadtxt('conserved_scores_albicans.txt')
    conserved_scores_albicans_foldseek  = conserved_scores_albicans_foldseek [:n_entries]
    print("file loaded, entries:", len(n_entries))

except:
    conserved_scores_albicans_foldseek = np.full(50, np.nan)

try:
    for i in range(n_entries,50):
        score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans = scores_evals_from_uniprot(conserved_genes_uniprot[i])
        print(score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans, conserved_genes[i], conserved_genes_uniprot[i])

        conserved_e_vals_cerevisiae_foldseek[i] = eval_cerevisiae
        conserved_scores_cerevisiae_foldseek[i] = score_cerevisiae
        conserved_e_vals_albicans_foldseek[i] = eval_albicans
        conserved_scores_albicans_foldseek[i] = score_albicans

except:
    for i in range(50):
        score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans = scores_evals_from_uniprot(conserved_genes_uniprot[i])
        print(score_cerevisiae, eval_cerevisiae, score_albicans, eval_albicans, conserved_genes[i], conserved_genes_uniprot[i])

        conserved_e_vals_cerevisiae_foldseek[i] = eval_cerevisiae
        conserved_scores_cerevisiae_foldseek[i] = score_cerevisiae
        conserved_e_vals_albicans_foldseek[i] = eval_albicans
        conserved_scores_albicans_foldseek[i] = score_albicans

np.savetxt('conserved_e_vals_cerevisiae_foldseek.txt', conserved_e_vals_cerevisiae_foldseek)
np.savetxt('conserved_scores_cerevisiae_foldseek.txt', conserved_scores_cerevisiae_foldseek)
np.savetxt('conserved_e_vals_albicans_foldseek.txt', conserved_e_vals_albicans_foldseek)
np.savetxt('conserved_scores_albicans_foldseek.txt', conserved_scores_albicans_foldseek)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  867k    0  867k    0     0  8030k      0 --:--:-- --:--:-- --:--:-- 8110k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.6M    0 22.6M    0     0  26.0M      0 --:--:-- --:--:-- --:--:-- 26.1M


0 100 0 100 SPBC336.07 Q9UST7


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  136k  100  136k    0     0  2152k      0 --:--:-- --:--:-- --:--:-- 2196k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  522k    0  522k    0     0  6226k      0 --:--:-- --:--:-- --:--:-- 6289k


no albicans
523 1.863e-21 0 100 SPCC1450.04 O74173


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241k  100  241k    0     0  3405k      0 --:--:-- --:--:-- --:--:-- 3448k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1567k    0 1567k    0     0  8726k      0 --:--:-- --:--:-- --:--:-- 8756k


...

waiting
no albicans
1603 1.067e-45 0 100 SPBC1706.03 Q9USY7


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  227k    0  227k    0     0  1784k      0 --:--:-- --:--:-- --:--:-- 1789k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1501k    0 1501k    0     0  9065k      0 --:--:-- --:--:-- --:--:-- 9098k


waiting
1384 1.314e-28 590 2.026e-13 SPAC688.09 Q9P6L7


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  341k    0  341k    0     0  2576k      0 --:--:-- --:--:-- --:--:-- 2585k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3494k    0 3494k    0     0  20.7M      0 --:--:-- --:--:-- --:--:-- 20.8M


no albicans
109 0.0005444 0 100 SPAC22H10.07 P40996


In [104]:
np.savetxt('conserved_e_vals_cerevisiae_foldseek.txt', conserved_e_vals_cerevisiae_foldseek)
np.savetxt('conserved_scores_cerevisiae_foldseek.txt', conserved_scores_cerevisiae_foldseek)
np.savetxt('conserved_e_vals_albicans_foldseek.txt', conserved_e_vals_albicans_foldseek)
np.savetxt('conserved_scores_albicans_foldseek.txt', conserved_scores_albicans_foldseek)

# Plotting the results



In [341]:
all_uniprot_ids = np.concatenate((unique_genes_uniprot, conserved_genes_uniprot[:50], unique_genes_uniprot, conserved_genes_uniprot[:50], unique_genes_uniprot, conserved_genes_uniprot[:50], unique_genes_uniprot, conserved_genes_uniprot[:50]))
all_gene_names = np.concatenate((unique_genes, conserved_genes[:50], unique_genes, conserved_genes[:50], unique_genes, conserved_genes[:50], unique_genes, conserved_genes[:50]))
all_scores = np.concatenate((unique_scores_cerevisiae, conserved_scores_cerevisiae[:50], unique_scores_cerevisiae_foldseek, conserved_scores_cerevisiae_foldseek[:50], unique_scores_albicans, conserved_scores_albicans[:50], unique_scores_albicans_foldseek, conserved_scores_albicans_foldseek[:50]))
all_e_vals = np.concatenate((unique_e_vals_cerevisiae, conserved_e_vals_cerevisiae[:50], unique_e_vals_cerevisiae_foldseek, conserved_e_vals_cerevisiae_foldseek[:50], unique_e_vals_albicans, conserved_e_vals_albicans[:50], unique_e_vals_albicans_foldseek, conserved_e_vals_albicans_foldseek[:50]))
unique_conserved = np.concatenate((np.full(len(unique_genes), 'unique'), np.full(50, 'conserved'), np.full(len(unique_genes), 'unique'), np.full(50, 'conserved'), np.full(len(unique_genes), 'unique'), np.full(50, 'conserved'), np.full(len(unique_genes), 'unique'), np.full(50, 'conserved')))
data_sources = np.concatenate((np.full(len(unique_genes), 'BlastP'), np.full(50, 'BlastP'), np.full(len(unique_genes), 'Foldseek'), np.full(50, 'Foldseek'), np.full(len(unique_genes), 'BlastP'), np.full(50, 'BlastP'), np.full(len(unique_genes), 'Foldseek'), np.full(50, 'Foldseek')))
#species S. cerevisiae or C. albicans
species = np.concatenate((np.full(len(unique_genes), 'S. cerevisiae'), np.full(50, 'S. cerevisiae'), np.full(len(unique_genes), 'S. cerevisiae'), np.full(50, 'S. cerevisiae'), np.full(len(unique_genes), 'C. albicans'), np.full(50, 'C. albicans'), np.full(len(unique_genes), 'C. albicans'), np.full(50, 'C. albicans')))
#create a df with uniprot IDs, gene names, scores and e-values, data_source, unique/conserved, species
df = pd.DataFrame({'uniprot_id': all_uniprot_ids, 'gene_name': all_gene_names, 'score': all_scores, 'e_value': all_e_vals, 'unique_conserved': unique_conserved, 'data_source': data_sources, 'species': species})


In [377]:
pombe_floc_dict = {"SPAC186.01":"pfl9",
                   "SPAC1F8.06":"pfl8",
                   "SPBC359.04c":"pfl7",
                   "SPAC977.07c":"pfl6",
                   "SPBC1289.15":"pfl5",
                   "SPCC188.09c":"pfl4",
                   "SPBC947.04":"pfl3",
                   "SPAP11E10.02c":"mam3",
                   "SPAPB15E9.01c":"pfl2",
                   "SPAPB2C8.01":"SPAPB2C8.01",
                   "SPBC1348.08c":"SPBC1348.08c",
                   "SPBC21D10.06c":"map4",
                   "SPBPJ4664.02":"SPBPJ4664.02",
                   "SPCC1742.01":"gsf2"}

df["color"] = ["True (n=14)" if x in pombe_floc_dict.keys() else "False" for x in df["gene_name"]]



In [378]:
#in df, for e_value turn Na into 100
df['e_value'] = df['e_value'].fillna(100)
#for score fill with 0
df['score'] = df['score'].fillna(0)
#any e_value higher than 100, turn into 100
df['e_value'] = df['e_value'].apply(lambda x: 100 if x > 100 else x)
#if lower than 10e-50, turn into 10e-50
df['e_value'] = df['e_value'].apply(lambda x: 10e-150 if x < 10e-150 else x)

In [398]:
import iqplot 
import bokeh.io
from scipy import stats
#output notebook
bokeh.io.output_notebook()
p = iqplot.stripbox(df, q= "score",cats = ["data_source","species", "unique_conserved"],color_column="color",q_axis="y",frame_width=500,frame_height=400, spread="jitter", jitter_kwargs={'width':0.6},
                    marker_kwargs=dict(size=8, alpha=0.5), box_kwargs=dict(line_color="black", line_width=2), whisker_kwargs=dict(line_color="black", line_width=2), median_kwargs=dict(line_color="black", line_width=2),show_legend=True)




#rotate x axis labels
p.xaxis.major_label_orientation = 1.2
#increa sizes of furhter x axis labels, like that of species and data source

p.xaxis.axis_label_text_font_size = '15pt'

p.xaxis.major_label_text_font_size = '15pt'
p.yaxis.axis_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '10pt'
    # no italic on labels
p.xaxis.axis_label_text_font_style = 'normal'
p.yaxis.axis_label_text_font_style = 'normal'

#y axis calles Score
p.yaxis.axis_label = 'Score'

#Legend name is Flocculin
p.legend.title = 'Flocculin genes'

    #remove legend
p.output_backend = "svg"


#print text on Mann whitney U p values between conserved and unique for each species and data source
p_val1= stats.mannwhitneyu(df[(df['unique_conserved'] == 'unique') & (df['species'] == 'S. cerevisiae') & (df['data_source'] == 'BlastP')]['score'], df[(df['unique_conserved'] == 'conserved') & (df['species'] == 'S. cerevisiae') & (df['data_source'] == 'BlastP')]['score'], alternative='two-sided')[1]
p_val2 = stats.mannwhitneyu(df[(df['unique_conserved'] == 'unique') & (df['species'] == 'S. cerevisiae') & (df['data_source'] == 'Foldseek')]['score'], df[(df['unique_conserved'] == 'conserved') & (df['species'] == 'S. cerevisiae') & (df['data_source'] == 'Foldseek')]['score'], alternative='two-sided')[1]
p_val3 = stats.mannwhitneyu(df[(df['unique_conserved'] == 'unique') & (df['species'] == 'C. albicans') & (df['data_source'] == 'BlastP')]['score'], df[(df['unique_conserved'] == 'conserved') & (df['species'] == 'C. albicans') & (df['data_source'] == 'BlastP')]['score'], alternative='two-sided')[1]
p_val4 = stats.mannwhitneyu(df[(df['unique_conserved'] == 'unique') & (df['species'] == 'C. albicans') & (df['data_source'] == 'Foldseek')]['score'], df[(df['unique_conserved'] == 'conserved') & (df['species'] == 'C. albicans') & (df['data_source'] == 'Foldseek')]['score'], alternative='two-sided')[1]


#print them on the plot in scientific notation to 3 significant figures
p.text(x=0.5, y=0.5, text=["p = {:.3e}".format(p_val1), "p = {:.3e}".format(p_val2), "p = {:.3e}".format(p_val3), "p = {:.3e}".format(p_val4)], text_font_size='10pt', text_align='center', text_baseline='middle', text_color='black', text_alpha=0.5)

p.y_range.end = 4000
    
bokeh.io.show(p)





Loading BokehJS ...

In [409]:
import bokeh.io
from bokeh.plotting import figure, show
from bokeh.models import Text
from scipy import stats
import iqplot
import pandas as pd

# Your DataFrame (df) and other data

# Create the plot
p = iqplot.stripbox(df, q="score", cats=["data_source", "species", "unique_conserved"], color_column="color", q_axis="y", frame_width=500, frame_height=400, spread="jitter", jitter_kwargs={'width': 0.6},
                    marker_kwargs=dict(size=8, alpha=0.5), box_kwargs=dict(line_color="black", line_width=2), whisker_kwargs=dict(line_color="black", line_width=2), median_kwargs=dict(line_color="black", line_width=2), show_legend=True)

# Customizations
p.xaxis.major_label_orientation = 1.2
p.xaxis.axis_label_text_font_size = '15pt'
p.xaxis.major_label_text_font_size = '15pt'
p.yaxis.axis_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '10pt'
p.xaxis.axis_label_text_font_style = 'normal'
p.yaxis.axis_label_text_font_style = 'normal'
p.yaxis.axis_label = 'Score'
p.legend.title = 'Flocculin genes'

# Calculate p-values
p_vals = []
species_data_sources = [
    ('C. albicans', 'BlastP'),
    ('S. cerevisiae', 'BlastP'),
    ('C. albicans', 'Foldseek'),
    ('S. cerevisiae', 'Foldseek')

]

for species, data_source in species_data_sources:
    unique_scores = df[(df['unique_conserved'] == 'unique') & (df['species'] == species) & (df['data_source'] == data_source)]['score']
    conserved_scores = df[(df['unique_conserved'] == 'conserved') & (df['species'] == species) & (df['data_source'] == data_source)]['score']
    p_val = stats.mannwhitneyu(unique_scores, conserved_scores, alternative='two-sided')[1]
    p_vals.append(p_val)

# Create the text labels
p_vals_labels = ["p = {:.3e}".format(p_val) for p_val in p_vals]

# Adjust text label positions
x_positions = [1.2,4,7,9.8]  # Corresponding to the species and data_source combinations
y_positions = [4100,4300,4100,4300]

# Add text labels to the plot
for x, y, label in zip(x_positions, y_positions, p_vals_labels):
    text = Text(x=x, y=y, text=[label], text_font_size='12pt', text_align='center', text_baseline='middle', text_color='black', text_alpha=1)
    p.add_glyph(text)

# Adjust y range
p.y_range.end = 4500

# Show the plot
bokeh.io.output_notebook()  # Use this if you're working in a Jupyter Notebook
show(p)


Loading BokehJS ...

In [415]:
#create a df called weird results. Include entries where unique yet e_val is less than 10-5 and conserved yet e_val is hgiher than 10-5
weird_results = df[((df['unique_conserved'] == 'unique') & (df['e_value'] < 10**-5)) | ((df['unique_conserved'] == 'conserved') & (df['e_value'] > 10**-5))]
#keep only blastp results
weird_results_blastp = weird_results[weird_results['data_source'] == 'BlastP']
#create weird_results_foldseek which is unique and e_value < 10-5 
weird_results_foldseek = df[(df['unique_conserved'] == 'unique') & (df['e_value'] < 10**-5) & (df['data_source'] == 'Foldseek')]
weird_results_foldseek


,uniprot_id,gene_name,score,e_value,unique_conserved,data_source,species,color
78,Q10211,SPAC4H3.03c,323.0,2.916000e-10,unique,Foldseek,S. cerevisiae,False


In [416]:
weird_results_blastp

,uniprot_id,gene_name,score,e_value,unique_conserved,data_source,species,color
8,Q9C0Y2,SPAPB2C8.01,166.0,5.326090e-10,unique,BlastP,S. cerevisiae,True (n=14)
10,Q9URU4,SPBC1289.15,279.0,2.866220e-23,unique,BlastP,S. cerevisiae,True (n=14)
12,O74308,SPBC15D4.02,142.0,7.382550e-08,unique,BlastP,S. cerevisiae,False
15,O43019,SPBC354.05c,191.0,8.832570e-14,unique,BlastP,S. cerevisiae,False
24,O94532,SPCC895.05,336.0,7.411690e-30,unique,BlastP,S. cerevisiae,False
60,O94355,SPBC428.06c,70.0,9.330000e+00,conserved,BlastP,S. cerevisiae,False
64,O43089,SPBC947.14c,50.0,1.000000e+02,conserved,BlastP,S. cerevisiae,False
67,Q1MTN3,SPBC336.15,66.0,1.000000e+02,conserved,BlastP,S. cerevisiae,False
69,Q9UUA4,SPBC409.21,66.0,2.293840e+01,conserved,BlastP,S. cerevisiae,False
158,Q9C0Y2,SPAPB2C8.01,250.0,7.473730e-20,unique,BlastP,C. albicans,True (n=14)


In [410]:
#create this subset df[(df['data_source']=='BlastP') & (df['unique_conserved']=='unique') & (df['species']=='S. cerevisiae')]
subset = df.loc[(df['data_source']=='BlastP') & (df['unique_conserved']=='conserved') & (df['species']=='C. albicans')]
#sort by score
subset = subset.sort_values(by=['score'])
subset

,uniprot_id,gene_name,score,e_value,unique_conserved,data_source,species,color
175,Q9UST7,SPBC336.07,0.0,1.000000e+02,conserved,BlastP,C. albicans,False
195,Q8WZK1,SPBC839.09c,0.0,1.000000e+02,conserved,BlastP,C. albicans,False
194,O60082,SPCC1494.08c,0.0,1.000000e+02,conserved,BlastP,C. albicans,False
219,Q9UUA4,SPBC409.21,54.0,1.000000e+02,conserved,BlastP,C. albicans,False
214,O43089,SPBC947.14c,58.0,4.604260e+01,conserved,BlastP,C. albicans,False
217,Q1MTN3,SPBC336.15,65.0,1.000000e+02,conserved,BlastP,C. albicans,False
198,O60154,SPBC19C7.05,69.0,4.418550e+00,conserved,BlastP,C. albicans,False
210,O94355,SPBC428.06c,115.0,1.649550e-05,conserved,BlastP,C. albicans,False
182,O60132,SPBC1706.01,122.0,3.960870e-05,conserved,BlastP,C. albicans,False
202,O59780,SPCC320.03,185.0,1.525340e-12,conserved,BlastP,C. albicans,False


In [279]:
subset


0       0.0
1      59.0
2       0.0
3      72.0
4      59.0
5       0.0
6     111.0
7      61.0
8     166.0
9      62.0
10    279.0
11     59.0
12    142.0
13     47.0
14      0.0
15    191.0
16     73.0
17      0.0
18      0.0
19    109.0
20      0.0
21    119.0
22      0.0
23     66.0
24    336.0
Name: score, dtype: float64

In [161]:
#df show C. albicans and BlastP
df[(df["species"]=="C. albicans") & (df["data_source"]=="BlastP") & (df["unique_conserved"]=="conserved")]


,uniprot_id,gene_name,score,e_value,unique_conserved,data_source,species
175,Q9UST7,SPBC336.07,52.0,2.810710e+01,conserved,BlastP,C. albicans
176,O74173,SPCC1450.04,445.0,6.737090e-57,conserved,BlastP,C. albicans
177,O13814,SPAC17H9.17c,399.0,2.304530e-45,conserved,BlastP,C. albicans
178,Q10251,SPAC56F8.03,116.0,6.282880e-07,conserved,BlastP,C. albicans
179,Q9USK1,SPCC4B3.11c,41.0,2.896700e+01,conserved,BlastP,C. albicans
180,O42858,SPAC23H3.05c,117.0,1.115810e-07,conserved,BlastP,C. albicans
181,Q92372,SPBC660.13c,52.0,1.195300e+01,conserved,BlastP,C. albicans
182,O60132,SPBC1706.01,63.0,7.862400e-01,conserved,BlastP,C. albicans
183,Q9UTG6,SPAC1805.09c,50.0,1.071830e+01,conserved,BlastP,C. albicans
184,P27638,SPAC31G5.09c,1141.0,6.192590e-157,conserved,BlastP,C. albicans


In [153]:
# select rows where data_source is Foldseek or BlastP
foldseek_rows = df[df['data_source'] == 'Foldseek']
blastp_rows = df[df['data_source'] == 'BlastP']

# merge the two dataframes on gene_name
merged_df = pd.merge(foldseek_rows, blastp_rows, on=['gene_name','species'], suffixes=('_foldseek', '_blastp'))

# select rows where the score difference is 1000 or more
score_diff = merged_df['score_foldseek'] - merged_df['score_blastp']
selected_rows = merged_df[abs(score_diff) >= 500]

selected_rows 

,uniprot_id_foldseek,gene_name,score_foldseek,e_value_foldseek,unique_conserved_foldseek,data_source_foldseek,species,uniprot_id_blastp,score_blastp,e_value_blastp,unique_conserved_blastp,data_source_blastp
27,O13814,SPAC17H9.17c,1208.0,1.398000e-32,conserved,Foldseek,S. cerevisiae,O13814,390.0,3.020440e-43,conserved,BlastP
28,Q10251,SPAC56F8.03,0.0,1.000000e+02,conserved,Foldseek,S. cerevisiae,Q10251,2231.0,0.000000e+00,conserved,BlastP
30,O42858,SPAC23H3.05c,1407.0,1.831000e-37,conserved,Foldseek,S. cerevisiae,O42858,569.0,1.530830e-69,conserved,BlastP
31,Q92372,SPBC660.13c,1789.0,2.035000e-65,conserved,Foldseek,S. cerevisiae,Q92372,1057.0,3.548990e-137,conserved,BlastP
34,P27638,SPAC31G5.09c,1853.0,3.646000e-44,conserved,Foldseek,S. cerevisiae,P27638,1090.0,3.882490e-149,conserved,BlastP
35,P40377,SPBC4.04c,3933.0,0.000000e+00,conserved,Foldseek,S. cerevisiae,P40377,2567.0,0.000000e+00,conserved,BlastP
36,O74946,SPCC553.09c,1738.0,5.362000e-55,conserved,Foldseek,S. cerevisiae,O74946,686.0,9.685900e-82,conserved,BlastP
39,Q9UUF1,SPBC17A3.08,1167.0,4.069000e-32,conserved,Foldseek,S. cerevisiae,Q9UUF1,561.0,1.044300e-69,conserved,BlastP
41,Q9P7V9,SPBC1703.12,0.0,1.000000e+02,conserved,Foldseek,S. cerevisiae,Q9P7V9,617.0,1.902360e-71,conserved,BlastP
42,O13651,SPBC8D2.06,6704.0,0.000000e+00,conserved,Foldseek,S. cerevisiae,O13651,3513.0,0.000000e+00,conserved,BlastP
